In [1]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()


Saving SDG1_worldbank_poverty_raw.csv to SDG1_worldbank_poverty_raw.csv


In [2]:
# Replace with your exact file name
df = pd.read_csv('SDG1_worldbank_poverty_raw.csv', skiprows=4)

# Drop irrelevant columns
df = df.drop(columns=['Indicator Name', 'Indicator Code'])

In [3]:
# Upload metadata file
uploaded = files.upload()

Saving Metadata_Country.csv to Metadata_Country.csv


In [4]:
# Load metadata
meta = pd.read_csv('Metadata_Country.csv')

# Check available regions (optional)
meta['Region'].unique()


array(['Latin America & Caribbean', nan, 'South Asia',
       'Sub-Saharan Africa', 'Europe & Central Asia',
       'Middle East & North Africa', 'East Asia & Pacific',
       'North America'], dtype=object)

In [5]:
# Merge poverty data with metadata on Country Code
merged_df = pd.merge(df, meta, how='left', on='Country Code')

# Filter to African countries (e.g., "Sub-Saharan Africa", "Middle East & North Africa")
africa_df = merged_df[merged_df['Region'].str.contains('Africa', na=False)]


In [6]:
## Check columns to see what was added from the merge
print(merged_df.columns)

Index(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023', '2024', 'Unnamed: 69', 'Region',
       'IncomeGroup', 'SpecialNotes', 'TableName', 'Unnamed: 5'],
      dtype='object')


In [7]:
# Drop metadata columns that exist (only if they exist)
columns_to_drop = ['Region', 'IncomeGroup', 'SpecialNotes', 'TableName']
for col in columns_to_drop:
    if col in merged_df.columns:
        merged_df = merged_df.drop(columns=col)

# Proceed with filtering African countries
africa_df = merged_df[merged_df['Country Code'].isin(
    meta[meta['Region'].str.contains('Africa', na=False)]['Country Code']
)]

# Melt from wide to long format
africa_long = africa_df.melt(id_vars=['Country Name', 'Country Code'],
                             var_name='Year',
                             value_name='Poverty Rate')

# Remove missing values
africa_long = africa_long.dropna()

# Convert year column to int (ignore non-numeric year values)
africa_long = africa_long[africa_long['Year'].str.isnumeric()]
africa_long['Year'] = africa_long['Year'].astype(int)

# Preview cleaned African dataset
africa_long.head()


,Country Name,Country Code,Year,Poverty Rate
1338,Israel,ISR,1979,0.4
1416,Madagascar,MDG,1980,61.7
1691,Morocco,MAR,1984,22.4
1731,Botswana,BWA,1985,50.1
1733,Cote d'Ivoire,CIV,1985,14.0


In [8]:
africa_long.to_csv('africa_poverty_cleaned.csv', index=False)
files.download('africa_poverty_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>